In [1]:
%load_ext lab_black

In [2]:
import json
import itertools

import numpy as np
import pandas as pd

from pathlib import Path

In [3]:
from mppsteel.config.reference_lists import RESOURCE_CATEGORY_MAPPER

from mppsteel.data_preprocessing.variable_plant_cost_archetypes import (
    plant_variable_costs,
    generate_feedstock_dict,
    generate_variable_costs,
    vc_mapper,
)

from mppsteel.utility.file_handling_utility import (
    read_pickle_folder,
    get_scenario_pkl_path,
)

from mppsteel.config.model_config import (
    PKL_DATA_FORMATTED,
    PKL_DATA_IMPORTS,
    MODEL_YEAR_RANGE,
)

In [4]:
plant_variable_costs

<function mppsteel.data_preprocessing.variable_plant_cost_archetypes.plant_variable_costs(scenario_dict: dict) -> pandas.core.frame.DataFrame>

# Generate Complete Result

In [4]:
scenario_path = Path("/tmp/scenario_dict.json")
with scenario_path.open("r") as f:
    scenario_dict = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/scenario_dict.json'

In [ ]:
scenario_dict

In [63]:
%%time
df_reference = plant_variable_costs(scenario_dict)

Variable Cost Loop: 100%|██████████████████████████████████████████████████████| 2356/2356 [03:16<00:00, 12.01it/s]


CPU times: user 3min 12s, sys: 5.25 s, total: 3min 17s
Wall time: 3min 18s


In [64]:
df_reference[df_reference.material_category == "Electricity"]

,technology,material_category,metric_type,unit,value,year,country_code,cost,cost_type
14,Avg BF-BOF,Electricity,Purchased energy,GJ/t steel,0.197855,2020,DZA,4.498158,Electricity
37,BAT BF-BOF,Electricity,Purchased energy,GJ/t steel,0.197855,2020,DZA,4.498158,Electricity
60,BAT BF-BOF_bio PCI,Electricity,Purchased energy,GJ/t steel,0.197855,2020,DZA,4.498158,Electricity
83,BAT BF-BOF_H2 PCI,Electricity,Purchased energy,GJ/t steel,0.197855,2020,DZA,4.498158,Electricity
106,BAT BF-BOF+CCUS,Electricity,Purchased energy,GJ/t steel,6.967080,2020,DZA,158.393830,Electricity
...,...,...,...,...,...,...,...,...,...
1192035,DRI-Melt-BOF,Electricity,Purchased energy,GJ/t steel,0.694297,2050,USA,11.717554,Electricity
1192058,DRI-Melt-BOF+CCUS,Electricity,Purchased energy,GJ/t steel,3.501320,2050,USA,59.091243,Electricity
1192081,DRI-Melt-BOF_100% zero-C H2,Electricity,Purchased energy,GJ/t steel,6.182434,2050,USA,104.340013,Electricity
1192104,Electrowinning-EAF,Electricity,Purchased energy,GJ/t steel,10.924412,2050,USA,184.369658,Electricity


In [73]:
query_str = "material_category == 'Electricity' and technology == 'DRI-Melt-BOF+CCUS' and year == 2050 and country_code == 'USA'"
df_reference.query(query_str)

,technology,material_category,metric_type,unit,value,year,country_code,cost,cost_type
1192058,DRI-Melt-BOF+CCUS,Electricity,Purchased energy,GJ/t steel,3.50132,2050,USA,59.091243,Electricity


In [18]:
intermediate_path = get_scenario_pkl_path(
    scenario_dict["scenario_name"], "intermediate"
)
eur_to_usd_rate = scenario_dict["eur_to_usd"]

steel_plants = read_pickle_folder(PKL_DATA_FORMATTED, "steel_plants_processed", "df")
steel_plant_region_ng_dict = (
    steel_plants[["country_code", "cheap_natural_gas"]]
    .set_index("country_code")
    .to_dict()["cheap_natural_gas"]
)
power_grid_prices_ref = read_pickle_folder(
    intermediate_path, "power_grid_prices_ref", "df"
)
h2_prices_ref = read_pickle_folder(intermediate_path, "h2_prices_ref", "df")
bio_model_prices_ref = read_pickle_folder(
    intermediate_path, "bio_model_prices_ref", "df"
)
ccs_model_transport_ref = read_pickle_folder(
    intermediate_path, "ccs_model_transport_ref", "df"
)
ccs_model_storage_ref = read_pickle_folder(
    intermediate_path, "ccs_model_storage_ref", "df"
)
business_cases = read_pickle_folder(
    PKL_DATA_FORMATTED, "standardised_business_cases", "df"
).reset_index()
static_energy_prices = read_pickle_folder(
    PKL_DATA_IMPORTS, "static_energy_prices", "df"
)[["Metric", "Year", "Value"]]
static_energy_prices.set_index(["Metric", "Year"], inplace=True)
feedstock_dict = generate_feedstock_dict(eur_to_usd_rate)
steel_plant_country_codes = list(steel_plants["country_code"].unique())

In [22]:
product_range_year_country = list(
    itertools.product(MODEL_YEAR_RANGE, steel_plant_country_codes)
)

In [24]:
len(product_range_year_country)

2356

In [25]:
product_range_year_country[0]

(2020, 'DZA')

In [ ]:
plant_variable_costs

In [28]:
%%time
year, country_code = product_range_year_country[0]
df = generate_variable_costs(
    year=year,
    country_code=country_code,
    business_cases_df=business_cases,
    ng_dict=steel_plant_region_ng_dict,
    feedstock_dict=feedstock_dict,
    static_energy_df=static_energy_prices,
    electricity_ref=power_grid_prices_ref,
    hydrogen_ref=h2_prices_ref,
    bio_ref=bio_model_prices_ref,
    ccs_storage_ref=ccs_model_storage_ref,
    ccs_transport_ref=ccs_model_transport_ref,
)

CPU times: user 89.8 ms, sys: 4.82 ms, total: 94.6 ms
Wall time: 94.5 ms


In [30]:
df.shape

(506, 8)

In [50]:
def vc_mapper_fast(
    row: pd.Series,
    country_code: str,
    year: int,
    static_year: int,
    electricity_ref: dict,
    hydrogen_ref: dict,
    bio_ref: dict,
    ccs_transport_ref: dict,
    ccs_storage_ref: dict,
    feedstock_dict: dict,
    static_energy_df: pd.DataFrame,
    ng_flag: int,
) -> float:
    """_summary_

    Args:
        row (pd.Series): A Series containing the consumption rates for each technology and resource.
        country_code (str): The country code to create a reference for.
        year (int): The year to create a reference for.
        static_year (int): The static year to be used for datasets that end after a certain year.
        electricity_ref (dict): The electricity grid price reference dict.
        hydrogen_ref (dict): The hydrogen price reference dict.
        bio_ref (dict): The bio price reference dict.
        ccs_transport_ref (dict): The ccs transport price reference dict.
        ccs_storage_ref (dict): The ccs storage price reference dict.
        feedstock_dict (dict): The feedstock price reference dict.
        static_energy_df (pd.DataFrame): The static energy reference dict.
        ng_flag (int): The natural gas flag price.

    Returns:
        float: A float value containing the variable cost value to assign.
    """
    if row.material_category == "Electricity":
        return row.value * electricity_ref[(year, country_code)]
    elif row.material_category == "Hydrogen":
        return row.value * hydrogen_ref[(year, country_code)]

    elif RESOURCE_CATEGORY_MAPPER[row.material_category] == "Bio Fuels":
        return row.value * bio_ref[(year, country_code)]

    elif RESOURCE_CATEGORY_MAPPER[row.material_category] == "CCS":
        return row.value * (
            ccs_transport_ref[country_code] + ccs_storage_ref[country_code]
        )

    elif (row.material_category == "Natural gas") and (ng_flag == 1):
        return (
            row.value * static_energy_df.loc["Natural gas - low", static_year]["Value"]
        )

    elif (row.material_category == "Natural gas") and (ng_flag == 0):
        return (
            row.value * static_energy_df.loc["Natural gas - high", static_year]["Value"]
        )

    elif row.material_category == "Plastic waste":
        return row.value * feedstock_dict["Plastic waste"]

    elif (RESOURCE_CATEGORY_MAPPER[row.material_category] == "Fossil Fuels") and (
        row.material_category not in ["Natural gas", "Plastic waste"]
    ):
        return (
            row.value
            * static_energy_df.loc[row.material_category, static_year]["Value"]
        )

    elif RESOURCE_CATEGORY_MAPPER[row.material_category] == "Feedstock":
        return row.value * feedstock_dict[row.material_category]

    elif RESOURCE_CATEGORY_MAPPER[row.material_category] == "Other Opex":
        if row.material_category in {"BF slag", "Other slag"}:
            return row.value * feedstock_dict[row.material_category]

        elif row.material_category == "Steam":
            return row.value * static_energy_df.loc["Steam", static_year]["Value"]
    return 0

In [60]:
%%time
business_cases_df = business_cases
ng_dict = steel_plant_region_ng_dict
electricity_ref = power_grid_prices_ref
hydrogen_ref = h2_prices_ref
bio_ref = bio_model_prices_ref
ccs_storage_ref = ccs_model_storage_ref
ccs_transport_ref = ccs_model_transport_ref
static_energy_df = static_energy_prices

df_c = business_cases_df.copy()
static_year = min(2026, year)
ng_flag = ng_dict[country_code]
df_c["year"] = year
df_c["country_code"] = country_code
df_c["cost"] = df_c.apply(
    vc_mapper_fast,
    country_code=country_code,
    year=year,
    static_year=static_year,
    electricity_ref=electricity_ref,
    hydrogen_ref=hydrogen_ref,
    bio_ref=bio_ref,
    ccs_transport_ref=ccs_transport_ref,
    ccs_storage_ref=ccs_storage_ref,
    feedstock_dict=feedstock_dict,
    static_energy_df=static_energy_df,
    ng_flag=ng_flag,
    axis=1,
)

CPU times: user 709 µs, sys: 154 µs, total: 863 µs
Wall time: 770 µs


In [61]:
df_c.shape

(506, 5)

In [55]:
df_c

,technology,material_category,metric_type,unit,value,year,country_code,cost
0,Avg BF-BOF,Iron ore,Feedstock,t/t steel,1.367188,2020,DZA,133.615284
1,Avg BF-BOF,Scrap,Feedstock,t/t steel,0.168300,2020,DZA,37.776618
2,Avg BF-BOF,DRI,Feedstock,t/t steel,0.000000,2020,DZA,0.000000
3,Avg BF-BOF,Met coal,Feedstock,t/t steel,0.367583,2020,DZA,45.131784
4,Avg BF-BOF,Coke,Fuel,GJ/t steel,7.437468,2020,DZA,0.000000
...,...,...,...,...,...,...,...,...
501,BAT BF-BOF+BECCUS,Process emissions,Other,tCO2/t steel,0.066590,2020,DZA,0.000000
502,BAT BF-BOF+BECCUS,Emissivity wout CCS,Other,tCO2/t steel,1.721016,2020,DZA,0.000000
503,BAT BF-BOF+BECCUS,Captured CO2,Other,tCO2/t steel,2.125104,2020,DZA,7.870818
504,BAT BF-BOF+BECCUS,Used CO2,Other,tCO2/t steel,0.000000,2020,DZA,0.000000


In [56]:
RESOURCE_CATEGORY_MAPPER

{'Iron ore': 'Feedstock',
 'Scrap': 'Feedstock',
 'DRI': 'Feedstock',
 'Met coal': 'Fossil Fuels',
 'Thermal coal': 'Fossil Fuels',
 'Coke': 'Fossil Fuels',
 'COG': 'Fossil Fuels',
 'BF gas': 'Fossil Fuels',
 'BOF gas': 'Fossil Fuels',
 'Natural gas': 'Fossil Fuels',
 'Plastic waste': 'Fossil Fuels',
 'Biomass': 'Bio Fuels',
 'Biomethane': 'Bio Fuels',
 'Hydrogen': 'Hydrogen',
 'Electricity': 'Electricity',
 'Steam': 'Other Opex',
 'BF slag': 'Other Opex',
 'Other slag': 'Other Opex',
 'Captured CO2': 'CCS',
 'Used CO2': 'CCS',
 'Process emissions': 'Emissivity',
 'Emissivity wout CCS': 'Emissivity',
 'Emissivity': 'Carbon Cost'}

In [57]:
506 * 2356

1192136